# 동물 전문가 시스템 구현하기

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners)의 예제입니다.

이 샘플에서는 몇 가지 신체적 특성에 기반하여 동물을 판별하는 간단한 지식 기반 시스템을 구현할 것입니다. 이 시스템은 다음과 같은 AND-OR 트리로 표현할 수 있습니다 (전체 트리의 일부이며, 더 많은 규칙을 쉽게 추가할 수 있습니다):

![](../../../../../../translated_images/ko/AND-OR-Tree.5592d2c70187f283.webp)


## 후방 추론을 사용하는 자체 전문가 시스템 셸

생산 규칙에 기반한 지식 표현을 위한 간단한 언어를 정의해 봅시다. 우리는 규칙을 정의하기 위해 Python 클래스를 키워드로 사용할 것입니다. 기본적으로 세 가지 유형의 클래스가 있습니다:
* `Ask`는 사용자에게 물어봐야 하는 질문을 나타냅니다. 가능한 답변 집합을 포함합니다.
* `If`는 규칙을 나타내며, 규칙 내용을 저장하기 위한 문법적 설탕 역할을 합니다.
* `AND`/`OR`는 트리의 AND/OR 분기를 나타내는 클래스입니다. 내부에 인수 목록을 저장합니다. 코드를 단순화하기 위해 모든 기능은 상위 클래스 `Content`에 정의되어 있습니다.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

우리 시스템에서 작업 기억은 **속성-값 쌍**으로 된 **사실들**의 목록을 포함합니다. 지식 베이스는 동작(작업 기억에 삽입되어야 하는 새로운 사실)을 조건에 매핑하는 하나의 큰 사전으로 정의할 수 있으며, 조건은 AND-OR 표현식으로 표현됩니다. 또한, 일부 사실은 `Ask`할 수 있습니다.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

역방향 추론을 수행하기 위해 `Knowledgebase` 클래스를 정의하겠습니다. 이 클래스는 다음을 포함합니다:
* 작동 중인 `memory` - 속성을 값에 매핑하는 딕셔너리
* 위에서 정의한 형식의 Knowledgebase `rules`

두 가지 주요 메서드는 다음과 같습니다:
* 필요한 경우 추론을 수행하여 속성의 값을 얻는 `get` 메서드입니다. 예를 들어, `get('color')`는 색상 슬롯의 값을 얻습니다(필요하면 묻고, 나중에 사용할 수 있도록 작동 메모리에 값을 저장합니다). `get('color:blue')`라고 묻는다면, 색상을 물어본 후 그 색상에 따라 `y`/`n` 값을 반환합니다.
* 실제 추론을 수행하는 `eval` 메서드로, AND/OR 트리를 탐색하고 하위 목표를 평가하는 등의 작업을 수행합니다.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

이제 동물 지식 기반을 정의하고 상담을 수행해 보겠습니다. 이 호출은 질문을 할 것입니다. 예-아니오 질문에는 `y`/`n`으로 대답할 수 있고, 다지선다형 질문에는 0부터 N까지 숫자를 지정하여 대답할 수 있습니다.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Experta를 사용한 순방향 추론

다음 예제에서는 지식 표현 라이브러리 중 하나인 [Experta](https://github.com/nilp0inter/experta)를 사용하여 순방향 추론을 구현해 보겠습니다. **Experta**는 파이썬에서 순방향 추론 시스템을 만들기 위한 라이브러리로, 고전적인 기존 시스템 [CLIPS](http://www.clipsrules.net/index.html)와 유사하게 설계되었습니다.

순방향 연결(forward chaining)을 직접 구현하는 것도 큰 문제 없이 가능했겠지만, 단순한 구현은 보통 효율적이지 않습니다. 보다 효과적인 규칙 매칭을 위해 특수한 알고리즘인 [Rete](https://en.wikipedia.org/wiki/Rete_algorithm)가 사용됩니다.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

우리는 시스템을 `KnowledgeEngine`을 상속하는 클래스로 정의할 것입니다. 각 규칙은 언제 규칙이 실행될지를 지정하는 `@Rule` 주석이 붙은 별도의 함수로 정의됩니다. 규칙 내부에서는 `declare` 함수를 사용하여 새로운 사실을 추가할 수 있으며, 이러한 사실을 추가하면 순방향 추론 엔진에 의해 더 많은 규칙이 호출됩니다.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

지식 기반을 정의한 후, 초기 사실 몇 가지로 작업 기억을 채우고 `run()` 메서드를 호출하여 추론을 수행합니다. 그 결과로, 새로운 추론된 사실들이 작업 기억에 추가되는 것을 볼 수 있으며, (모든 초기 사실들을 올바르게 설정했다면) 동물에 관한 최종 사실도 포함됩니다.


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**면책 조항**:  
본 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있음을 양지해 주시기 바랍니다. 원문 문서는 권위 있는 공식 자료로 간주되어야 합니다. 중요한 정보에 대해서는 전문적인 인간 번역을 권장합니다. 본 번역의 사용으로 인한 오해나 잘못된 해석에 대해서는 당사가 책임지지 않습니다.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
